In [1]:
import json
with open("scopus.config") as handle:
    config = json.load(handle)

In [2]:
import pandas as pd
xls = pd.ExcelFile("../../data/polish-list/full-list-20230718.xlsx")

In [3]:
df = pd.read_excel(xls, 
                   sheet_name=config['sheet_name'], 
                   header=config['header']
                   )

Imported everything. Marking "x" in column "703" (ID of IT category) indicates the IT tag of object.

In [60]:
refined = df[df[config['category']] == 'x'][df['Punkty'] > config['points_treshold']]
search_list = pd.concat([refined[e] for e in config['extract']]).dropna().unique()
assert len(search_list) == len(refined)

/tmp/ipykernel_28070/844489044.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  refined = df[df[config['category']] == 'x'][df['Punkty'] > config['points_treshold']]


In [61]:
def make_query(elements, keyword, conjugator):
    answer = keyword + "("
    for e in elements:
        answer += f"{e} {conjugator} "
    answer = answer[:-(len(conjugator) + 1)]
    answer += ")"
    return answer

In [62]:
with open('scopus.answer', "w") as handle:
    handle.write(make_query(search_list, config['keyword'], config['conjugator']))

In [64]:
from pybliometrics.scopus import ScopusSearch
lt = []
for x in search_list:
    q = make_query([x], config['keyword'], config['conjugator'])
    try:
        s = ScopusSearch(q,view ="STANDARD", download=False, subscriber=False, verbose=False)
        lt.append(s.get_results_size())
    except Exception as e:
        e = str(e)
        lt.append(int(e.split(" ")[1].replace(',', '')))

/home/richard/.local/lib/python3.8/site-packages/urllib3/poolmanager.py:315: DeprecationWarning: The 'strict' parameter is no longer needed on Python 3+. This will raise an error in urllib3 v2.1.0.
  warnings.warn(


In [71]:
def write_output(search_list, config, out_file):
    with open(out_file, "a") as handle:
        handle.write(make_query(search_list, config['keyword'], config['conjugator']))
        handle.write('\n')
    return

In [73]:
x = []
obj = 0
for s, l in zip(search_list, lt):
    if obj + l < 20000:
        x.append(s)
        obj += l
    elif l > 20000:
        print(s, ":  ", l)
    else:
        write_output(x, config, 'scopus.queries')
        x = [s]
        obj = l
write_output(x, config, 'scopus.queries')

0018-926X :   23963
0305-1048 :   49176


Data clearing

In [4]:
import os
import pathlib
p = pathlib.Path("/mnt/c/Users/richa/Downloads")
files = [x for x in os.listdir(p) if x[-4:] == ".csv"]
files = [os.path.join(p, x) for x in files]

In [5]:
df1 = pd.read_csv(files[0], usecols=['EID', 'Funding Texts'])
df2 = pd.read_csv(files[1], usecols=['EID', 'Funding Texts'])

In [13]:
df = pd.concat([pd.read_csv(f, usecols=['DOI', 'Funding Texts']).dropna() for f in files])

/home/richard/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3318: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):
/home/richard/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3318: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


In [30]:
df.drop_duplicates(subset=['DOI'], keep='first').to_csv('/home/richard/Naukowe/papiery/001-ACK-EXTRACTION/PAPER-ACK-EXTRACTION/data/scraped-scopus/base.csv', index=False)